In [ ]:
print('hello')

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# データ読み取り

In [ ]:
# === セルA: CSV から df_full を読み込む ===
import pandas as pd
import numpy as np

# 必要に応じてファイル名を変更してOK
DF_FULL_PATH = "data/creativity_df_full.csv"

df_full = pd.read_csv(DF_FULL_PATH)

# もし index 列が一緒に出力されていた場合は削除
if "Unnamed: 0" in df_full.columns:
    df_full = df_full.drop(columns=["Unnamed: 0"])

print("df_full shape:", df_full.shape)
print("df_full columns (head):", df_full.columns[:15])

# スコア部分の簡単な確認
print("\n===== Score describe() (from CSV) =====")
print(df_full[["creativity", "originality", "coherence", "creativity_score"]].describe())

In [ ]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 解析用配列を抜き出す

In [ ]:
# === セルB: EOS / GEN 特徴量とスコアを numpy に復元 ===

# 列名から、自動で EOS / GEN の埋め込み部分を抽出
eos_cols = [c for c in df_full.columns if c.startswith("eos_dim_")]
gen_cols = [c for c in df_full.columns if c.startswith("gen_dim_")]

print("num eos dims:", len(eos_cols))
print("num gen dims:", len(gen_cols))

# 特徴量行列（float32）
X_eos = df_full[eos_cols].to_numpy(dtype=np.float32)   # [N, D]
X_gen = df_full[gen_cols].to_numpy(dtype=np.float32)   # [N, D]

# スコア（ここでは creativity_score をメインに）
y_crea_score = df_full["creativity_score"].astype(np.float32).to_numpy()
y_crea       = df_full["creativity"].astype(np.float32).to_numpy()
y_orig       = df_full["originality"].astype(np.float32).to_numpy()
y_coh        = df_full["coherence"].astype(np.float32).to_numpy()

print("X_eos shape:", X_eos.shape)
print("X_gen shape:", X_gen.shape)
print("y_crea_score shape:", y_crea_score.shape)

# GEN 最終層から creative_direction を再計算（full & train/test）

In [ ]:
# === セルC: GEN 最終層から creative_direction を再計算 ===
from sklearn.model_selection import train_test_split

X = X_gen
y = y_crea_score  # ここでは creativity_score をターゲットに

# 1) full データで High / Low を定義
high_th = np.quantile(y, 0.7)
low_th  = np.quantile(y, 0.3)

mask_high = y >= high_th
mask_low  = y <= low_th

X_high = X[mask_high]
X_low  = X[mask_low]

print("X_high shape:", X_high.shape)
print("X_low  shape:", X_low.shape)

mu_high = X_high.mean(axis=0)
mu_low  = X_low.mean(axis=0)

creative_direction_gen_full = mu_high - mu_low
creative_direction_gen_full = creative_direction_gen_full / (np.linalg.norm(creative_direction_gen_full) + 1e-8)

proj_full = X @ creative_direction_gen_full
corr_full = np.corrcoef(proj_full, y)[0, 1]
print("=== GEN direction: full データでの creativity_score との相関 ===")
print("corr(full):", corr_full)

# 2) train / test split で汎化性チェック
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

high_th_tr = np.quantile(y_train, 0.7)
low_th_tr  = np.quantile(y_train, 0.3)

mask_high_tr = y_train >= high_th_tr
mask_low_tr  = y_train <= low_th_tr

Xh_tr = X_train[mask_high_tr]
Xl_tr = X_train[mask_low_tr]

mu_h_tr = Xh_tr.mean(axis=0)
mu_l_tr = Xl_tr.mean(axis=0)

creative_direction_gen_tr = mu_h_tr - mu_l_tr
creative_direction_gen_tr = creative_direction_gen_tr / (np.linalg.norm(creative_direction_gen_tr) + 1e-8)

proj_test = X_test @ creative_direction_gen_tr
corr_test = np.corrcoef(proj_test, y_test)[0, 1]

print("=== GEN direction: test データでの creativity_score との相関 ===")
print("corr(test):", corr_test)

# projection と各スコア（creativity / originality / coherence）の相関

In [ ]:
# === セルD: projection × 各スコアの相関 ===

def corr(a, b):
    return float(np.corrcoef(a, b)[0, 1])

proj = proj_full  # セルCで計算した full データ用 projection

# NaN ガード（ほぼ出ないと思うけど念のため）
mask_valid = ~np.isnan(proj)

proj_valid = proj[mask_valid]
crea_valid = y_crea[mask_valid]
orig_valid = y_orig[mask_valid]
coh_valid  = y_coh[mask_valid]
score_valid= y_crea_score[mask_valid]

print("===== Projection × 各スコアの相関 =====")
print(f"creativity          : {corr(proj_valid, crea_valid):.4f}")
print(f"originality         : {corr(proj_valid, orig_valid):.4f}")
print(f"coherence           : {corr(proj_valid, coh_valid):.4f}")
print(f"creativity_score(*) : {corr(proj_valid, score_valid):.4f}")

# スコア分布と簡単なヒストグラム（全部 CSV から）

In [ ]:
# === セルE: スコアの基本統計と分布ヒスト ===
import matplotlib.pyplot as plt

print("===== Score describe() =====")
print(df_full[["creativity", "originality", "coherence", "creativity_score"]].describe())

for col in ["creativity", "originality", "coherence", "creativity_score"]:
    plt.figure()
    df_full[col].hist(bins=10)
    plt.title(col)
    plt.xlabel(col)
    plt.ylabel("count")
    plt.show()

# EOS ベースの direction も試したい場合（オプション）

In [ ]:
# === セルF: EOS 埋め込み版の creative_direction（オプション） ===

X = X_eos
y = y_crea_score

high_th = np.quantile(y, 0.7)
low_th  = np.quantile(y, 0.3)

mask_high = y >= high_th
mask_low  = y <= low_th

X_high = X[mask_high]
X_low  = X[mask_low]

mu_high = X_high.mean(axis=0)
mu_low  = X_low.mean(axis=0)

creative_direction_eos = mu_high - mu_low
creative_direction_eos = creative_direction_eos / (np.linalg.norm(creative_direction_eos) + 1e-8)

proj_eos = X @ creative_direction_eos
corr_eos = np.corrcoef(proj_eos, y)[0, 1]

print("=== EOS direction: full データでの creativity_score との相関 ===")
print("corr(full, EOS):", corr_eos)

# GEN埋め込み → creativity_score を予測する LassoCV

In [ ]:
# === セルL1: GEN 埋め込みで creativity_score を予測する LassoCV ===
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np

# もしまだなら（さっきのセルBで定義済みなら不要）
gen_cols = [c for c in df_full.columns if c.startswith("gen_dim_")]
X_gen = df_full[gen_cols].to_numpy(dtype=np.float32)
y_crea_score = df_full["creativity_score"].astype(np.float32).to_numpy()

# train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X_gen, y_crea_score, test_size=0.3, random_state=42
)

# スケーリング + LassoCV（α自動チューニング）
lasso_gen = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", LassoCV(
        alphas=np.logspace(-3, 1, 20),  # 必要なら範囲を変えてOK
        cv=5,
        random_state=42,
        n_jobs=-1,
    )),
])

lasso_gen.fit(X_train, y_train)

alpha_opt = lasso_gen.named_steps["lasso"].alpha_
print("最適 alpha:", alpha_opt)

# 性能チェック
y_train_pred = lasso_gen.predict(X_train)
y_test_pred  = lasso_gen.predict(X_test)

def corr(a, b):
    return float(np.corrcoef(a, b)[0, 1])

print("\n=== Lasso (GEN) performance ===")
print(f"Train R^2      : {lasso_gen.score(X_train, y_train):.4f}")
print(f"Test  R^2      : {lasso_gen.score(X_test,  y_test):.4f}")
print(f"Train corr     : {corr(y_train, y_train_pred):.4f}")
print(f"Test  corr     : {corr(y_test,  y_test_pred):.4f}")
print(f"y_train std    : {np.std(y_train):.4f}")
print(f"y_test  std    : {np.std(y_test):.4f}")

# どの GEN 次元が効いているか（重要次元の可視化）

In [ ]:
# === セルL2: Lasso 係数から重要次元を可視化 ===
import pandas as pd
import numpy as np

lasso = lasso_gen.named_steps["lasso"]
coef = lasso.coef_  # shape: [hidden_dim]

# ゼロでない係数の数
non_zero = np.sum(coef != 0)
print("非ゼロ係数の数:", non_zero, "/", coef.size)

# 係数を DataFrame 化してソート
coef_df = pd.DataFrame({
    "dim_index": np.arange(coef.size),
    "col_name": gen_cols,
    "coef": coef,
    "abs_coef": np.abs(coef),
}).sort_values("abs_coef", ascending=False)

print("\n=== 上位 20 次元（絶対値の大きい係数）=== ")
display(coef_df.head(20))

print("\n=== 下位 20 次元（ほぼ効いていない次元）=== ")
display(coef_df.tail(20))

# creativity / originality / coherence それぞれを Lasso で見る（任意）

In [ ]:
# === セルL3: 各スコア（crea / orig / coh）ごとに LassoCV を回す ===
targets = {
    "creativity":   df_full["creativity"].astype(np.float32).to_numpy(),
    "originality":  df_full["originality"].astype(np.float32).to_numpy(),
    "coherence":    df_full["coherence"].astype(np.float32).to_numpy(),
}

lasso_results = {}

for name, y in targets.items():
    X_train, X_test, y_train, y_test = train_test_split(
        X_gen, y, test_size=0.3, random_state=42
    )

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", LassoCV(
            alphas=np.logspace(-3, 1, 20),
            cv=5,
            random_state=42,
            n_jobs=-1,
        )),
    ])

    pipe.fit(X_train, y_train)
    lasso = pipe.named_steps["lasso"]

    y_train_pred = pipe.predict(X_train)
    y_test_pred  = pipe.predict(X_test)

    lasso_results[name] = {
        "model": pipe,
        "coef": lasso.coef_,
        "alpha": lasso.alpha_,
        "train_r2": pipe.score(X_train, y_train),
        "test_r2":  pipe.score(X_test,  y_test),
        "train_corr": corr(y_train, y_train_pred),
        "test_corr":  corr(y_test,  y_test_pred),
        "non_zero": int(np.sum(lasso.coef_ != 0)),
    }

# 結果まとめ
print("=== Lasso summary by target ===")
for name, info in lasso_results.items():
    print(f"\n[{name}]")
    print(f"  alpha      : {info['alpha']:.4e}")
    print(f"  non_zero   : {info['non_zero']} / {len(info['coef'])}")
    print(f"  Train R^2  : {info['train_r2']:.4f}")
    print(f"  Test  R^2  : {info['test_r2']:.4f}")
    print(f"  Train corr : {info['train_corr']:.4f}")
    print(f"  Test  corr : {info['test_corr']:.4f}")

# 特定ターゲットの「重要次元ランキング」を見る（任意）

In [ ]:
# === セルL4: 好きなターゲットの重要次元ランキングを見る ===
target_name = "originality"  # "creativity" / "originality" / "coherence" から選ぶ

info = lasso_results[target_name]
coef = info["coef"]

coef_df_target = pd.DataFrame({
    "dim_index": np.arange(coef.size),
    "col_name": gen_cols,
    "coef": coef,
    "abs_coef": np.abs(coef),
}).sort_values("abs_coef", ascending=False)

print(f"=== {target_name} に効いている GEN 次元 Top 20 ===")
display(coef_df_target.head(20))